In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import pytorch_lightning as pl

from DGCNN import DGCNN_reg
from utils.data_depth import DepthDataset
from utils.dataloader import DataLoader
from utils.plot import plot_3d_pointcloud

### Hyperparameters:

In [4]:
batch_size = 12

k = 20
emb_dims = 1024
dropout = 0.5

lr = 1e-3
epochs = 1000

### Datasets and dataloaders:

In [5]:
train_set    = DepthDataset(partition='train', outputs=['depth','feature_map'], normalize=True, pc_mean=0.5, repeat=1)
train_loader = DataLoader(train_set, num_workers=8, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
valid_set    = DepthDataset(partition='valid', outputs=['depth','feature_map'], normalize=True, pc_mean=0.5, repeat=1)
valid_loader = DataLoader(valid_set, num_workers=8, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

In [6]:
pixels, depth, fm = train_set[0]
plot_3d_pointcloud(train_set.denormalize(pixels), depth, train_set.im_shape)

C:\Users\Gastón\Desktop\Gaston\CONICET\repos\dgcnn_emma\utils\plot.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[fx, 0, px, 0.],


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0033710…

### Pytorch lightning system:

In [5]:
class DepthDGCNN(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.dgcnn = DGCNN_reg(k=k, emb_dims=emb_dims, dropout=dropout, output_channels=1)
        
    def forward(self, batch):
        pixels, depth, feature_map = batch
        depth_pred = self.dgcnn(pixels, feature_map)
        return depth_pred
    
    def training_step(self, batch, batch_idx):
        pixels, depth, feature_map = batch
        depth_pred = self.dgcnn(pixels, feature_map)
        loss = F.mse_loss(depth_pred, depth)
        self.log('loss', {'train': loss}, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        pixels, depth, feature_map = batch
        depth_pred = self.dgcnn(pixels, feature_map)
        loss = F.mse_loss(depth_pred, depth)
        self.log('loss', {'val': loss}, on_step=False, on_epoch=True)
        
    def test_step(self, batch, batch_idx):
        pixels, depth, feature_map = batch
        depth_pred = self.dgcnn(pixels, feature_map)
        loss = F.mse_loss(depth_pred, depth)
        self.log('loss', {'val': loss}, on_step=False, on_epoch=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer

### Init tensorboard

In [6]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 14752), started 1:09:32 ago. (Use '!kill 14752' to kill it.)

### Init and train the model

In [6]:
dgcnn = DepthDGCNN()

trainer = pl.Trainer(gpus=1, max_epochs=epochs, precision=32)

trainer.fit(dgcnn, train_loader, valid_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | dgcnn | DGCNN_reg | 979 K 
------------------------------------
979 K     Trainable params
0         Non-trainable params
979 K     Total params
3.918     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

### Test model

In [10]:
# Create test loader
test_loader = DataLoader(train_set, num_workers=1, batch_size=1, shuffle=False, drop_last=False, pin_memory=True)
test_loader_iter = iter(test_loader)

# Send model to device
device = torch.device("cuda")
dgcnn.to(device);

#trainer.test(test_dataloaders=val)

In [16]:
# Get next pointcloud and send it to device
pixels, depth, fn = next(test_loader_iter)
pixels, fn = pixels.to(device), fn.to(device)

# Predict
depth_pred = dgcnn((pixels, depth, fn))

# Send prediction to cpu and plot it
pixels = pixels.detach().cpu().squeeze().numpy()
depth_pred = depth_pred.detach().cpu().squeeze().numpy()
plot_3d_pointcloud(train_set.denormalize(pixels), depth_pred, train_set.im_shape)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0079492…

### Load model

In [ ]:
checkpoint_dir = 'lightning_logs/version_xx/checkpoints/'

filename = listdir(checkpoint_dir)[0]
model = DepthDGCNN.load_from_checkpoint(checkpoint_dir + filename)